In [1]:
%reload_kedro

[02/14/24 15:35:43] INFO     Resolved project path as: /scratch/rahit/modspy-data.                  ]8;id=350934;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=905047;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Resolved project path as: /scratch/rahit/modspy-data.                  ]8;id=384287;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=709939;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[02/14/24 15:35:44] INFO     Kedro project ModSpy-Data                                              ]8;id=825017;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=409716;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Kedro project ModSpy-Data                                              ]8;id=493011;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=564606;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=863235;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=712938;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=643694;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=550041;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

In [2]:
import itertools
import logging
from collections import defaultdict

from tqdm import tqdm
from rich import print
from IPython.display import display, HTML

import dask
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, progress, performance_report
from dask_jobqueue import SLURMCluster

import torch
from torch_geometric.data import HeteroData

In [84]:
cluster = SLURMCluster(
    cores=32,  # Number of cores per job
    processes=1,
    memory='200GB',  # Memory allocated to each worker
    walltime='02:30:00',  # Walltime limit for each job
    # Specify any additional SLURM or Dask configurations as needed
)

[02/15/24 03:03:17] INFO     State start                                                          ]8;id=261582;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=270223;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#1615\1615]8;;\

                    INFO       Scheduler at:  tcp://10.29.85.222:44122                            ]8;id=699289;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=579460;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3860\3860]8;;\

                    INFO       dashboard at:                     :8787                            ]8;id=97054;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=870031;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3862\3862]8;;\

In [85]:
client = Client(cluster)  # start distributed scheduler locally.

[02/15/24 03:03:20] INFO     Receive client connection:                                           ]8;id=971219;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=527533;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5228\5228]8;;\
                             Client-aeea8569-cbd8-11ee-99d9-3cfdfe750910                                           

                    INFO     Starting established connection to tcp://10.29.85.222:58494                ]8;id=861182;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=469323;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#867\867]8;;\

In [86]:
display(cluster)
display(client)

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.29.85.222:8787/status,
Dashboard: http://10.29.85.222:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.29.85.222:44122,Workers: 0
Dashboard: http://10.29.85.222:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [87]:
# Scale the cluster to the desired number of workers
cluster.scale(jobs=4)  # Request 10 jobs, adjust based on your needs

In [57]:
!pwd

/scratch/rahit/modspy-data/notebooks/monarch_gnn


In [60]:
nodes_df = dd.read_csv('../../data/01_raw/monarch/monarch-kg_nodes.tsv', sep='\t', usecols=['id', 'category', 'name', 'in_taxon', 'in_taxon_label', 'symbol'], dtype={'id':'object', 
                        'category':'object', 'name':'object', 'in_taxon':'object', 'in_taxon_label':'object', 'symbol':'object'})
edges_df = dd.read_parquet('../../data/02_intermediate/monarch/edges_with_node_cat')

In [61]:
# Group by 'node_type' and assign type-wise indices
nodes_df['type_index'] = nodes_df.groupby('category').cumcount()

In [62]:
nodes_df.head()

,id,category,name,in_taxon,in_taxon_label,symbol,type_index
0,PomBase:SPAC1002.01,biolink:Gene,mrx11,NCBITaxon:4896,Schizosaccharomyces pombe,mrx11,0
1,PomBase:SPAC1002.02,biolink:Gene,pom34,NCBITaxon:4896,Schizosaccharomyces pombe,pom34,1
2,PomBase:SPAC1002.03c,biolink:Gene,gls2,NCBITaxon:4896,Schizosaccharomyces pombe,gls2,2
3,PomBase:SPAC1002.04c,biolink:Gene,taf11,NCBITaxon:4896,Schizosaccharomyces pombe,taf11,3
4,PomBase:SPAC1002.05c,biolink:Gene,jmj2,NCBITaxon:4896,Schizosaccharomyces pombe,jmj2,4


In [63]:
# nodes_df = nodes_df.assign(enum_idx=1)
# nodes_df['enum_idx'] = nodes_df['enum_idx'].cumsum() - 1
nodes_df = nodes_df.set_index('id')
nodes_df.columns

Index(['category', 'name', 'in_taxon', 'in_taxon_label', 'symbol',
       'type_index'],
      dtype='object')

In [64]:
len(nodes_df.index.unique())

862115

In [65]:
len(nodes_df['type_index'].unique())

559272

In [66]:
############# CHANGE ME #######################
edges_df = edges_df.rename(columns={'subject_category': 'e_category', 'edge_category': 'subject_category'}).rename(columns={'e_category': 'edge_category'})
###############################################
_edf = edges_df.merge(nodes_df, left_on='subject', right_index=True, suffixes=('_ndf', '_edf'))
# print(f"Columns after merging on subject category: {_edf.columns}")
_edf = _edf.rename(columns={'type_index': 'subject_id'})
print(f"Columns after merging on subject category: {_edf.columns}")
display(_edf.head())

_edf = _edf.merge(nodes_df, left_on='object', right_index=True, suffixes=('_ndf', '_edf'))
# print(f"Columns after merging on object category: {_edf.columns}")
_edf = _edf.rename(columns={'type_index': 'object_id'})
print(f"Columns after merging on object category: {_edf.columns}")
display(_edf.head())

Columns after merging on subject category: Index(['id', 'subject', 'edge_category', 'predicate', 
'subject_category',
       'object_category', 'object', 'category', 'name', 'in_taxon',
       'in_taxon_label', 'symbol', 'subject_id'],
      dtype='object')

,id,subject,edge_category,predicate,subject_category,object_category,object,category,name,in_taxon,in_taxon_label,symbol,subject_id
0,uuid:6ef055fb-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846
1,uuid:6ef055fc-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846
2,uuid:a77a4a50-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846
3,uuid:c3ee0928-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846
4,uuid:c9de5a17-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846


Columns after merging on object category: Index(['id', 'subject', 'edge_category', 'predicate', 'subject_category',
       'object_category', 'object', 'category_ndf', 'name_ndf', 'in_taxon_ndf',
       'in_taxon_label_ndf', 'symbol_ndf', 'subject_id', 'category_edf',
       'name_edf', 'in_taxon_edf', 'in_taxon_label_edf', 'symbol_edf',
       'object_id'],
      dtype='object')

,id,subject,edge_category,predicate,subject_category,object_category,object,category_ndf,name_ndf,in_taxon_ndf,in_taxon_label_ndf,symbol_ndf,subject_id,category_edf,name_edf,in_taxon_edf,in_taxon_label_edf,symbol_edf,object_id
0,uuid:6ef055fb-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846,biolink:Gene,BCL10,NCBITaxon:9606,Homo sapiens,BCL10,517786
1,uuid:6ef055fc-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846,biolink:Gene,BCL10,NCBITaxon:9606,Homo sapiens,BCL10,517786
2,uuid:a77a4a50-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846,biolink:Gene,BCL10,NCBITaxon:9606,Homo sapiens,BCL10,517786
3,uuid:c3ee0928-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846,biolink:Gene,BCL10,NCBITaxon:9606,Homo sapiens,BCL10,517786
4,uuid:c9de5a17-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,biolink:PairwiseGeneToGeneInteraction,biolink:interacts_with,biolink:Gene,biolink:Gene,HGNC:989,biolink:Gene,CARD9,NCBITaxon:9606,Homo sapiens,CARD9,518846,biolink:Gene,BCL10,NCBITaxon:9606,Homo sapiens,BCL10,517786


In [21]:
len(_edf['subject'].unique().compute())

492073

In [22]:
len(edges_df['subject'].unique().compute())

492073

In [31]:
len(_edf['subject_id'].unique().compute())

492073

In [19]:
_edf.shape[0].compute()

11412471

In [20]:
edges_df.shape[0].compute()

11412471

In [43]:
nodes_df['category'].unique().values.compute()

array(['biolink:PhenotypicFeature', 'biolink:NamedThing',
       'biolink:BiologicalProcessOrActivity', 'biolink:PhenotypicQuality',
       'biolink:AnatomicalEntity', 'biolink:OrganismalEntity',
       'biolink:MolecularEntity', 'biolink:ChemicalEntity',
       'biolink:SmallMolecule', 'biolink:Protein', 'biolink:Drug',
       'biolink:RNAProduct', 'biolink:ConfidenceLevel', 'biolink:Cell',
       'biolink:CellularComponent', 'biolink:MacromolecularComplex',
       'biolink:CellLine', 'biolink:OrganismTaxon', 'biolink:Dataset',
       'biolink:Disease', 'biolink:Gene', 'biolink:GeographicExposure',
       'biolink:Procedure', 'biolink:Transcript', 'biolink:EvidenceType',
       'biolink:InformationContentEntity', 'biolink:DrugExposure',
       'biolink:ChemicalExposure', 'biolink:GrossAnatomicalStructure',
       'biolink:EnvironmentalFeature', 'biolink:EnvironmentalProcess',
       'biolink:Invertebrate', 'biolink:Fungus', 'biolink:Mammal',
       'biolink:Zygosity', 'biolink:Genetic

In [67]:
edges = _edf[['id','subject', 'subject_id', 'subject_category', 'predicate', 'edge_category', 'object_category', 'object_id', 'object']]
print(f"Columns after renaming and triming: {edges.columns}")

Columns after renaming and triming: Index(['id', 'subject', 'subject_id', 'subject_category', 'predicate',
       'edge_category', 'object_category', 'object_id', 'object'],
      dtype='object')

In [28]:
def custome(df):
    
    pass

_edf = edges.groupby(['subject_category','predicate','object_category'])['subject_id','object_id'] #.apply(custom)
_edf.get_group(2) #.compute()
# edges.head(n=3)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/pandas/core/indexes/base.py:3621 in get_loc  │
│                                                                                                  │
│   3618 │   │   │   │   )                                                                         │
│   3619 │   │   │   casted_key = self._maybe_cast_indexer(key)                                    │
│   3620 │   │   │   try:                                                                          │
│ ❱ 3621 │   │   │   │   return self._engine.get_loc(casted_key)                                   │
│   3622 │   │   │   except KeyError as err:                                                       │
│   3623 │   │   │   │   raise KeyError(key) from err                                              │
│   3624 │   │   │   except TypeError:                                                             │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:136                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:163                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5198                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5206                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: ('subject_id', 'object_id')

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3 │   pass                                                                                     │
│   4                                                                                              │
│   5 _edf = edges.groupby(['subject_category','predicate','object_category'])['subject_id','o     │
│ ❱ 6 _edf.get_group(2) #.compute()                                                                │
│   7 # edges.head(n=3)                                                                            │
│   8                                                                                              │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/dataframe/groupby.py:1968 in get_group  │
│                                                                                                  │
│   1965 │   │                                                                                     │
│   1966 │   │   meta = self._meta.obj                                                             │
│   1967 │   │   if is_dataframe_like(meta) and self._slice is not None:                           │
│ ❱ 1968 │   │   │   meta = meta[self._slice]                                                      │
│   1969 │   │   columns = meta.columns if is_dataframe_like(meta) else meta.name                  │
│   1970 │   │                                                                                     │
│   1971 │   │   return map_partitions(                                                            │
│                                                    

## grouped_size = _edf.size.compute()

In [83]:
grouped_edge = _edf['subject_category','predicate','object_category'].unique().compute()
for key, edge_info in grouped_edge:
    print(key)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 grouped_edge = _edf['subject_category','predicate','object_category'].unique().compute()     │
│   2 for key, edge_info in grouped_edge:                                                          │
│   3 │   print(key)                                                                               │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'SeriesGroupBy' object is not subscriptable

In [63]:
# Function to apply to each partition
def create_edge_tuple(df):
    df['edge_key'] = list(zip(df['subject_category'], df['predicate'], df['object_category']))
    return df

# Apply the function to each partition
# __e = __e.map_partitions(create_edge_tuple)
__edf = __e.map_partitions(create_edge_tuple).compute()
display(__edf.head())


# edges_df['edge_key'] = dd.zip(edges_df['subject_category'], edges_df['predicate'], edges_df['object_category'])

[02/12/24 01:25:04] INFO     Register worker <WorkerState 'tcp://10.29.84.84:33024', name:        ]8;id=486065;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=470951;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4212\4212]8;;\
                             SLURMCluster-3, status: init, memory: 0, processing: 0>                               

                    INFO     Starting worker compute stream, tcp://10.29.84.84:33024              ]8;id=545040;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=759321;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5468\5468]8;;\

                    INFO     Starting established connection to tcp://10.29.84.84:36664                 ]8;id=263360;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=604666;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#867\867]8;;\

,id,subject,subject_id,subject_category,predicate,edge_category,object_category,object_id,object,edge_key
750522,uuid:a08bb3ea-9bab-11ee-b780-6b2918cfaf31,HGNC:21911,562720,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,541256,HGNC:17012,"(biolink:Gene, biolink:interacts_with, biolink..."
739442,uuid:c18f21af-9bab-11ee-b780-6b2918cfaf31,HGNC:30595,563475,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,555052,HGNC:6903,"(biolink:Gene, biolink:interacts_with, biolink..."
3339584,uuid:ab1eab9a-9ba8-11ee-b780-6b2918cfaf31,ZFIN:ZDB-GENE-040718-356,19418,biolink:Gene,biolink:expressed_in,biolink:GeneToExpressionSiteAssociation,biolink:GrossAnatomicalStructure,822506,ZFA:0001094,"(biolink:Gene, biolink:expressed_in, biolink:G..."
3016944,uuid:1a75c6ed-9bb5-11ee-b780-6b2918cfaf31,NCBIGene:396145,306308,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,305657,NCBIGene:395432,"(biolink:Gene, biolink:interacts_with, biolink..."
215609,uuid:f29aebd7-9bad-11ee-b780-6b2918cfaf31,NCBIGene:282322,434384,biolink:Gene,biolink:enables,biolink:MacromolecularMachineToMolecularActivi...,biolink:BiologicalProcessOrActivity,645268,GO:0019904,"(biolink:Gene, biolink:enables, biolink:Biolog..."


In [67]:
# Function to apply to each partition
def create_edge_tuple(df):
    df['edge_key'] = list(zip(df['subject_category'], df['predicate'], df['object_category']))
    return df

# Apply the function to each partition
_temp_e = __e.map_partitions(create_edge_tuple)[['edge_key', 'subject_id','object_id']]
_temp_bag = _temp_e.to_bag(format='dict')
_temp_bag.compute()

[{'edge_key': ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'),
  'subject_id': 562720,
  'object_id': 541256},
 {'edge_key': ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'),
  'subject_id': 563475,
  'object_id': 555052},
 {'edge_key': ('biolink:Gene',
   'biolink:expressed_in',
   'biolink:GrossAnatomicalStructure'),
  'subject_id': 19418,
  'object_id': 822506},
 {'edge_key': ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'),
  'subject_id': 306308,
  'object_id': 305657},
 {'edge_key': ('biolink:Gene',
   'biolink:enables',
   'biolink:BiologicalProcessOrActivity'),
  'subject_id': 434384,
  'object_id': 645268},
 {'edge_key': ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'),
  'subject_id': 570287,
  'object_id': 571213},
 {'edge_key': ('biolink:Gene',
   'biolink:enables',
   'biolink:BiologicalProcessOrActivity'),
  'subject_id': 565123,
  'object_id': 632006},
 {'edge_key': ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'),
  'su

[02/12/24 01:54:33] INFO     Connection to tcp://10.29.82.83:50410 has been closed.                     ]8;id=648339;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=333216;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.82.83:33504', name:          ]8;id=340306;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=279533;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-4, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.82.83:33504                                 ]8;id=485497;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=761463;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

[02/12/24 01:55:03] INFO     Connection to tcp://10.29.84.84:36664 has been closed.                     ]8;id=424939;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=915674;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.84.84:33024', name:          ]8;id=252931;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=860119;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-3, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.84.84:33024                                 ]8;id=950208;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=821610;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Lost all workers                                                     ]8;id=299194;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=80847;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4875\4875]8;;\

In [69]:

# Function to apply to each partition
def create_edge_tuple(df):
    df['edge_key'] = list(zip(df['subject_category'], df['predicate'], df['object_category']))
    return df

# Create the key of the edge_index
def key_function(edge):
    return edge['edge_key']

# accumulate the edges for a specific key
def binop(accumulator, edge):
    # accumulator is a list of (subject_id, object_id) tuples
    # edge is the current item being processed
    accumulator.append([edge['subject_id'], edge['object_id']])
    return accumulator

# combine multiple list that are accumulated
def combine(accumulator1, accumulator2):
    # Combine the lists from two accumulators
    return accumulator1 + accumulator2


# Apply the function to each partition
_temp_e = __e.map_partitions(create_edge_tuple)[['edge_key', 'subject_id','object_id']]
_temp_bag = _temp_e.to_bag(format='dict')
# Using foldby to group by 'edge_key' and consolidate (subject_id, object_id) pairs
_r = _temp_bag.foldby(key=key_function, binop=binop, combine=combine, initial=[])




# Convert to dictionary to get the final structure
_fr = dict(_r.compute())


In [70]:
_fr

{('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'): [(562720, 541256),
  (563475, 555052),
  (19418, 822506),
  (306308, 305657),
  (434384, 645268),
  (570287, 571213),
  (565123, 632006),
  (156507, 186689),
  (136035, 636090),
  (26875, 6936),
  (560117, 569358),
  (265323, 265124),
  (290860, 289440),
  (159477, 159476),
  (204661, 645174),
  (19540, 822506),
  (550388, 682580),
  (544262, 439730),
  (82593, 82586),
  (7542, 643338),
  (550335, 545291),
  (194292, 285516),
  (542763, 537931),
  (213222, 738457),
  (564191, 579538),
  (95332, 96114),
  (24145, 822506),
  (416377, 758628),
  (148439, 147814),
  (2331, 623387),
  (151577, 517351),
  (581430, 511526),
  (802425, 802233),
  (577911, 549204),
  (564798, 545308),
  (6565, 822506),
  (197136, 642243),
  (296832, 634187),
  (229721, 636485),
  (238176, 635546),
  (214868, 205819),
  (252075, 238326),
  (433680, 434189),
  (177165, 632711),
  (52361, 47347),
  (555253, 555221),
  (135507, 657549),
  (540555, 580255)

[02/12/24 02:33:22] INFO     Connection to tcp://10.29.81.228:39380 has been closed.                    ]8;id=813937;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=392122;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.81.228:42041', name:         ]8;id=954001;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=577772;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-3, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.81.228:42041                                ]8;id=983461;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=578707;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Connection to tcp://10.29.84.84:44888 has been closed.                     ]8;id=722328;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=642166;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.84.84:40354', name:          ]8;id=716085;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=40403;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-4, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.84.84:40354                                 ]8;id=817498;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=258657;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Lost all workers                                                     ]8;id=300549;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=997716;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4875\4875]8;;\

In [ ]:
_temp_e = __e.map_partitions(create_edge_tuple)[['edge_key', 'subject_id','object_id']]

In [ ]:
_temp_e

In [40]:
edges['subject_category'].unique().compute()

[02/12/24 00:52:44] ERROR    Task exception was never retrieved                                 ]8;id=975119;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/asyncio/base_events.py\base_events.py]8;;\:]8;id=340972;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/asyncio/base_events.py#1707\1707]8;;\
                             future: <Task finished name='Task-1034315'                                            
                             coro=<Client._gather.<locals>.wait() done, defined at                                 
                             /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/cl                    
                             ient.py:2134> exception=AllExit()>                                                    
                             ╭────────────── Traceback (most recent call last) ───────────────╮                    
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distribute │                    
                             │ d/client.py:2143 in wait                                       │                    
                             │                                                                │                    
                             │   2140 │   │   │   else:                                       │                    
                             │   2141 │   │   │   │   await st.wait()                         │                    
                             │   2142 │   │   │   if st.status != "finished" and errors == "r │                    
                             │ ❱ 2143 │   │   │   │   raise AllExit()                         │                    
                             │   2144 │   │                                                   │                    
                             │   2145 │   │   while True:                                     │                    
                             │   2146 │   │   │   logger.debug("Waiting on futures to clear b │                    
                             ╰────────────────────────────────────────────────────────────────╯                    
                             AllExit                                                                               

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 edges['subject_category'].unique().compute()                                                 │
│   2                                                                                              │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/base.py:314 in compute                  │
│                                                                                                  │
│    311 │   │   --------                                                                          │
│    312 │   │   dask.base.compute                                                                 │
│    313 │   │   """                                                                               │
│ ❱  314 │   │   (result,) = compute(self, traverse=False, **kwargs)                               │
│    315 │   │   return result                                                                     │
│    316 │                                                                                         │
│    317 │   def __await__(self):                                                                  │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/base.py:599 in compute                  │
│                                                                                                  │
│    596 │   │   keys.append(x.__dask_keys__())                                                    │
│    597 │   │   postcomputes.append(x.__dask_postcompute__())                                     │
│    598 │                                                                                         │
│ ❱  599 │   results = schedule(dsk, keys, **kwargs)                                               │
│    600 │   return repack([f(r, *a) for r, (f, a) in zip(results, postcomputes)])                 │
│    601                                                                                           │
│    602                                                                                           │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/client.py:3137 in get            │
│                                                                                                  │
│   3134 │   │   │   │   except Exception:                                                         │
│   3135 │   │   │   │   │   should_rejoin = False                                                 │
│   3136 │   │   │   try:                                                                          │
│ ❱ 3137 │   │   │   │   results = self.gather(packed, asynchronous=asynchronous, direct=direct)   │
│   3138 │   │   │   finally:                                                                      │
│   3139 │   │   │   │   for f in futures.values():                                                │
│   3140 │   │   │   │   │   f.release()                                                           │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/client.py:2306 in gather         │
│                                                                                                  │
│   2303 │   │   │   │   local_worker = thread_state.execution_state["worker"]                     │
│   2304 │   │   │   else:                                   

In [84]:

# Initialize HeteroData for the heterogeneous graph
data = HeteroData()

In [85]:
# Prepare node mapping and node types
# node_mapping = {node_id: i for i, node_id in enumerate(nodes_df['id'].unique())}
node_types = nodes_df['category'].unique()

# Add nodes to the graph
for node_type in tqdm(node_types):
    # get nodes of type `node_type`
    mask = nodes_df['category'] == node_type
    type_nodes = nodes_df[mask].compute()
    
    # node_features = torch.ones((type_nodes.shape[0], 1))
    data[node_type].num_nodes = type_nodes.shape[0] # node_features # torch.tensor(type_nodes.index.to_list(), dtype=torch.long).view(len(type_nodes.index.to_list()),1)

    # Add dummy features (e.g., a simple constant feature)
    data[node_type].x = torch.ones((data[node_type].num_nodes, 1))  # Each user has a feature vector of [1]



100%|██████████| 88/88 [04:22<00:00,  2.98s/it]


In [98]:
__e = edges.sample(frac=0.0001)
__e.shape[0].compute()

1141

In [18]:
__e.compute()

,id,subject,subject_id,subject_category,predicate,edge_category,object_category,object_id,object
6158261,uuid:a9ca20a5-9bb4-11ee-b780-6b2918cfaf31,FB:FBgn0026373,283734,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,304300,FB:FBgn0283478
5868283,uuid:c6ebb543-9ba8-11ee-b780-6b2918cfaf31,MGI:95841,237728,biolink:Gene,biolink:expressed_in,biolink:GeneToExpressionSiteAssociation,biolink:GrossAnatomicalStructure,32,EMAPA:16099
3818660,uuid:c83c398b-9bad-11ee-b780-6b2918cfaf31,RGD:1305649,194095,biolink:Gene,biolink:actively_involved_in,biolink:MacromolecularMachineToBiologicalProce...,biolink:BiologicalProcessOrActivity,17838,GO:0045663
1680312,uuid:73c15680-9bb0-11ee-b780-6b2918cfaf31,HGNC:11582,553852,biolink:Gene,biolink:has_phenotype,biolink:GeneToPhenotypicFeatureAssociation,biolink:PhenotypicFeature,9217,HP:0002079
2671903,uuid:417dfc0e-9bb6-11ee-b780-6b2918cfaf31,HGNC:14275,536376,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,548847,HGNC:10348
...,...,...,...,...,...,...,...,...,...
2507403,uuid:abb9701d-9ba8-11ee-b780-6b2918cfaf31,ZFIN:ZDB-GENE-010919-2,5591,biolink:Gene,biolink:expressed_in,biolink:GeneToExpressionSiteAssociation,biolink:GrossAnatomicalStructure,28306,ZFA:0007022
981752,uuid:02e2b4a0-9bb6-11ee-b780-6b2918cfaf31,RGD:1305513,185237,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,188860,RGD:620754
1229786,uuid:fa507149-9bb4-11ee-b780-6b2918cfaf31,Xenbase:XB-GENE-1007374,52513,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,48704,Xenbase:XB-GENE-970920
554587,uuid:bf217726-9bb4-11ee-b780-6b2918cfaf31,ZFIN:ZDB-GENE-040808-64,19633,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,8350,ZFIN:ZDB-GENE-050417-399


[02/14/24 19:36:31] INFO     Connection to tcp://10.29.85.181:41674 has been closed.                    ]8;id=619863;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=831670;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.85.181:38127', name:         ]8;id=796637;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=903740;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-3, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.85.181:38127                                ]8;id=16960;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=746382;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Connection to tcp://10.29.83.40:35798 has been closed.                     ]8;id=327980;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=63606;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

...

                    INFO     Removing comms to tcp://10.29.83.131:43973                                ]8;id=891949;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=100025;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Connection to tcp://10.29.84.46:44152 has been closed.                     ]8;id=827521;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=741135;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.84.46:46303', name:          ]8;id=347291;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=556104;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-2, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.84.46:46303                                 ]8;id=20003;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=222356;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Lost all workers                                                     ]8;id=243666;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=123622;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4875\4875]8;;\

In [93]:
# Scale the cluster to the desired number of workers
cluster.scale(jobs=1)  # Request 10 jobs, adjust based on your needs

In [ ]:
__e['edge_key'] = da

In [18]:

# Prepare edge types and mappings efficiently using Dask
edge_types = edges['edge_category'].unique().compute()  # Compute edge types on scheduler

def group_and_create_edge_index(df):
    edge_type_mappings = defaultdict(list)
    for _, row in df.iterrows():
        edge_key = (row['subject_category'], row['predicate'], row['object_category'])
        # print(edge_key)
        # print((row['subject_id'], row['object_id']))
        edge_type_mappings[edge_key].append((row['subject_id'], row['object_id']))

    results = []
    # Create edge index for each edge type within partitions
    for edge_key, edge_indices in edge_type_mappings.items():
        # print(edge_key)
        # print(edge_indices)
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
        # logging.debug(edge_key, edge_index)
        # yield edge_key, edge_index
        results.append((edge_key, edge_index))
    return results  # Return a list of tuples instead of a generator

# # Parallelize edge index creation across partitions
# edge_index_futures = edges.map_partitions(group_and_create_edge_index, meta=('edge_key', object))
# # edge_index_futures = edges.map_partitions(group_and_create_edge_index, meta=('x', object))

# result = edge_index_futures.compute()

# Gather results, ensuring proper edge_key ordering
# edge_index_dict = dict(result)


# Gemini
import dask.bag as db

bag = db.from_delayed(db.from_sequence(edges.map_partitions(group_and_create_edge_index, meta=('edge_key', object))))



[02/11/24 23:07:41] INFO     Connection to tcp://10.29.86.16:51020 has been closed.                     ]8;id=205465;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=722118;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.86.16:36819', name:          ]8;id=742053;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=716731;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-0, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.86.16:36819                                 ]8;id=898915;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=333954;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

[02/11/24 23:20:55] INFO     Connection to tcp://10.29.86.19:54760 has been closed.                     ]8;id=699321;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=595582;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.86.19:40403', name:          ]8;id=903185;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=678622;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-2, status: running, memory: 4, processing: 2>                            

...

                    INFO     Remove worker <WorkerState 'tcp://10.29.85.219:44991', name:         ]8;id=888103;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=175159;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-3, status: running, memory: 1, processing: 6>                            

                    INFO     Removing comms to tcp://10.29.85.219:44991                                ]8;id=160466;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=967960;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Lost all workers                                                     ]8;id=215939;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=160013;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4875\4875]8;;\

[02/11/24 23:22:59] ERROR    Task exception was never retrieved                                 ]8;id=641559;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/asyncio/base_events.py\base_events.py]8;;\:]8;id=824807;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/asyncio/base_events.py#1707\1707]8;;\
                             future: <Task finished name='Task-535739'                                             
                             coro=<Client._gather.<locals>.wait() done, defined at                                 
                             /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/cl                    
                             ient.py:2134> exception=AllExit()>                                                    
                             ╭────────────── Traceback (most recent call last) ───────────────╮                    
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distribute │                    
                             │ d/client.py:2143 in wait                                       │                    
                             │                                                                │                    
                             │   2140 │   │   │   else:                                       │                    
                             │   2141 │   │   │   │   await st.wait()                         │                    
                             │   2142 │   │   │   if st.status != "finished" and errors == "r │                    
                             │ ❱ 2143 │   │   │   │   raise AllExit()                         │                    
                             │   2144 │   │                                                   │                    
                             │   2145 │   │   while True:                                     │                    
                             │   2146 │   │   │   logger.debug("Waiting on futures to clear b │                    
                             ╰────────────────────────────────────────────────────────────────╯                    
                             AllExit                                                                               

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:36                                                                                   │
│                                                                                                  │
│   33 # Gemini                                                                                    │
│   34 import dask.bag as db                                                                       │
│   35                                                                                             │
│ ❱ 36 bag = db.from_delayed(db.from_sequence(edges.map_partitions(group_and_create_edge_index,    │
│   37                                                                                             │
│   38                                                                                             │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/bag/core.py:1770 in from_sequence       │
│                                                                                                  │
│   1767 │   --------                                                                              │
│   1768 │   read_text: Create bag from text files                                                 │
│   1769 │   """                                                                                   │
│ ❱ 1770 │   seq = list(seq)                                                                       │
│   1771 │   if npartitions and not partition_size:                                                │
│   1772 │   │   partition_size = int(math.ceil(len(seq) / npartitions))                           │
│   1773 │   if npartitions is None and partition_size is None:                                    │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/dataframe/core.py:720 in __len__        │
│                                                                                                  │
│    717 │   │   )                                                                                 │
│    718 │                                                                                         │
│    719 │   def __len__(self):                                                                    │
│ ❱  720 │   │   return self.reduction(                                                            │
│    721 │   │   │   len, np.sum, token="len", meta=int, split_every=False                         │
│    722 │   │   ).compute()                                                                       │
│    723                                                                                           │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/base.py:314 in compute                  │
│                                                                                                  │
│    311 │   │   --------                                                                          │
│    312 │   │   dask.base.compute                                                                 │
│    313 │   │   """                                                                               │
│ ❱  314 │   │   (result,) = compute(self, traverse=False, **kwargs)                               │
│    315 │   │   return result                                                                     │
│    316 │                                                                                         │
│    317 │   def __await__(self):                                                                  │
│                                                            

In [35]:
edge_index_dict

{0: <generator object group_and_create_edge_index at 0x2af52a977c10>,
 1: <generator object group_and_create_edge_index at 0x2af5309ad2e0>,
 2: <generator object group_and_create_edge_index at 0x2af5387ebd60>}

[02/10/24 04:38:56] INFO     Connection to tcp://10.29.81.51:54582 has been closed.                     ]8;id=416412;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=41278;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.81.51:42348', name:          ]8;id=687025;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=45590;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-0, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.81.51:42348                                 ]8;id=402654;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=959896;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Lost all workers                                                     ]8;id=378245;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=921514;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4875\4875]8;;\

In [22]:
for edge_key, edge_index in edge_index_dict.items():
    print(edge_key)
    print(edge_index[0])
    # data[edge_index[0]].edge_index = edge_index

0

(
    ('biolink:PairwiseGeneToGeneInteraction', 'biolink:interacts_with', 'biolink:Gene'),
    tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])
)

1

(
    ('biolink:GeneToExpressionSiteAssociation', 'biolink:expressed_in', 'biolink:GrossAnatomicalStructure'),
    tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
)

2

(
    ('biolink:GeneToExpressionSiteAssociation', 'biolink:expressed_in', 'biolink:GrossAnatomicalStructure'),
    tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
)

In [ ]:

# Prepare edge types and mappings
edge_types = edges['edge_category'].unique()
edge_type_mappings = defaultdict(list)

for _, row in edges.iterrows():
    edge_key = (row['subject_category', row['predicate'], row['object_category'])
    edge_type_mappings[edge_key].append((row['subject_id'], row['object_id']))

# Add edges to the graph
for edge_key, edge_indices in edge_type_mappings.items():
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    data[edge_key].edge_index = edge_index

In [ ]:
def map_nodes(df, node_mapping):
    # Map 'subject' and 'object' columns to their indices using the `node_mapping`
    df['subject_idx'] = df['subject'].map(node_mapping)
    df['object_idx'] = df['object'].map(node_mapping)
    return df[['subject_category', 'predicate', 'object_category', 'subject_idx', 'object_idx']]

edges_mapped = edges_df.map_partitions(map_nodes, node_mapping=node_mapping, meta=edges_df)


In [ ]:
def aggregate_edges(df):
    # Group by edge type and aggregate indices into lists
    grouped = df.groupby(['subject_category', 'predicate', 'object_category'])
    aggregated = grouped.apply(lambda x: list(zip(x['subject_idx'], x['object_idx'])), meta=('edges', 'object'))
    return aggregated.reset_index().rename(columns={0: 'edge_indices'})

edges_aggregated = edges_mapped.map_partitions(aggregate_edges, meta=edges_mapped)


In [23]:
import torch

In [14]:
node_mapping = {node_id: i for i, node_id in enumerate(nodes_df['id'].unique())}

In [24]:

edge_groups = edges.groupby(['subject_category', 'edge_category', 'object_category'])

def process_edge_group(group):
    subject_indices = group['subject_id'].compute()
    object_indices = group['object_id'].compute()
    edge_indices = torch.stack([subject_indices, object_indices], dim=0)
    # attributes = group[['attributes']].compute() if 'attributes' in group else None  # Load attributes with compute() if needed
    return dict(edge_index=edge_indices.contiguous())

# edge_data = dict(dask.dataframe.map_partitions(process_edge_group, edge_groups, meta={'edge_index': (torch.tensor, [None, None], torch.long))
#                 .compute())
                                                                                      
edge_data = dict(edge_groups.apply(process_edge_group, meta={'edge_index': (torch.tensor, [None, None], torch.long)}))
# edge_data = dict(map(process_edge_group, edge_groups))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/dataframe/dispatch.py:130 in make_meta  │
│                                                                                                  │
│   127 │   │   return x._meta                                                                     │
│   128 │                                                                                          │
│   129 │   try:                                                                                   │
│ ❱ 130 │   │   return make_meta_dispatch(x, index=index)                                          │
│   131 │   except TypeError:                                                                      │
│   132 │   │   if parent_meta is not None:                                                        │
│   133 │   │   │   func = make_meta_obj.dispatch(type(parent_meta))                               │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/utils.py:640 in __call__                │
│                                                                                                  │
│    637 │   │   """                                                                               │
│    638 │   │   Call the corresponding method based on type of argument.                          │
│    639 │   │   """                                                                               │
│ ❱  640 │   │   meth = self.dispatch(type(arg))                                                   │
│    641 │   │   return meth(arg, *args, **kwargs)                                                 │
│    642 │                                                                                         │
│    643 │   @property                                                                             │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/utils.py:634 in dispatch                │
│                                                                                                  │
│    631 │   │   │   │   register()                                                                │
│    632 │   │   │   │   self._lazy.pop(toplevel, None)                                            │
│    633 │   │   │   │   return self.dispatch(cls)  # recurse                                      │
│ ❱  634 │   │   raise TypeError(f"No dispatch for {cls}")                                         │
│    635 │                                                                                         │
│    636 │   def __call__(self, arg, *args, **kwargs):                                             │
│    637 │   │   """                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: No dispatch for <class 'dict'>

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 # edge_data = dict(dask.dataframe.map_partitions(process_edge_group, edge_groups, meta={    │
│   11 #                 .compute())                                                               │
│   12                                                                                             │
│ ❱ 13 edge_data = dict(edge_groups.apply(process_edge_group, meta={'edge_index': (torch.tensor    │
│   14 # edge_data = dict(map(process_edge_group, 

[02/10/24 04:01:20] INFO     Connection to tcp://10.29.81.51:45968 has been closed.                     ]8;id=162141;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=28076;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.81.51:37459', name:          ]8;id=303924;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=433699;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-0, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.81.51:37459                                 ]8;id=94241;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=355171;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Lost all workers                                                     ]8;id=409215;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=391763;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4875\4875]8;;\

In [ ]:
# Prepare node mapping and node types
node_mapping = {node_id: i for i, node_id in enumerate(nodes_df['id'].unique())}
node_types = nodes_df['category'].unique()

# # Initialize HeteroData for the heterogeneous graph
# data = HeteroData()

# # Add nodes to the graph
# for node_type in node_types:
#     # get nodes of type `node_type`
#     mask = nodes_df['category'] == node_type
#     type_nodes = nodes_df[mask]
    
#     # node_features = torch.ones((type_nodes.shape[0], 1))
#     data[node_type].num_nodes = type_nodes.shape[0] # node_features # torch.tensor(type_nodes.index.to_list(), dtype=torch.long).view(len(type_nodes.index.to_list()),1)

#     # Add dummy features (e.g., a simple constant feature)
#     data[node_type].x = torch.ones((data[node_type].num_nodes, 1))  # Each user has a feature vector of [1]


# Define a function to apply to each partition of the DataFrame
def process_partition(partition, node_mapping):
    edge_type_mappings = defaultdict(list)
    for _, row in partition.iterrows():
        edge_key = (row['subject_category', row['predicate'], row['object_category'])
        edge_type_mappings[edge_key].append((node_mapping[row['subject']], node_mapping[row['object']]))
    return edge_type_mappings

# Use map_partitions to apply the function to each partition
results = edges_df.map_partitions(process_partition, node_mapping=node_mapping, meta='object').compute()

# Combine the results from all partitions
combined_edge_type_mappings = defaultdict(list)
for result in results:
    for key, value in result.items():
        combined_edge_type_mappings[key].extend(value)

# Convert edge mappings to PyTorch tensors and add to HeteroData
data = HeteroData()
for edge_key, edge_indices in combined_edge_type_mappings.items():
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    data[edge_key].edge_index = edge_index

In [68]:
################## INITIALIZE NODES ##################

# Prepare node mapping and node types
node_mapping = {node_id: i for i, node_id in enumerate(nodes_df.index.unique())}
node_types = nodes_df['category'].unique()

# Initialize HeteroData for the heterogeneous graph
data = HeteroData()
    
# Prepare node mapping and node types
# node_mapping = {node_id: i for i, node_id in enumerate(nodes_df['id'].unique())}
node_types = nodes_df['category'].unique()

# Add nodes to the graph
for node_type in tqdm(node_types):
    # get nodes of type `node_type`
    mask = nodes_df['category'] == node_type
    type_nodes = nodes_df[mask].compute()
    
    # node_features = torch.ones((type_nodes.shape[0], 1))
    data[node_type].num_nodes = type_nodes.shape[0] # node_features # torch.tensor(type_nodes.index.to_list(), dtype=torch.long).view(len(type_nodes.index.to_list()),1)

    # Add dummy features (e.g., a simple constant feature)
    data[node_type].x = torch.ones((data[node_type].num_nodes, 1))  # Each user has a feature vector of [1]



100%|██████████| 88/88 [05:13<00:00,  3.57s/it]


In [97]:
cluster.scale(jobs=1)

[02/15/24 03:58:19] INFO     Register worker <WorkerState 'tcp://10.29.85.253:35930', name:       ]8;id=575937;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=626903;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4212\4212]8;;\
                             SLURMCluster-5, status: init, memory: 0, processing: 0>                               

                    INFO     Starting worker compute stream, tcp://10.29.85.253:35930             ]8;id=693511;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=220062;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5468\5468]8;;\

                    INFO     Starting established connection to tcp://10.29.85.253:49454                ]8;id=615271;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=333362;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#867\867]8;;\

In [99]:
edges = __e.copy()

# Convert the edge categories to categoricals for efficiency
edges['subject_category'] = edges['subject_category'].astype('category')
edges['predicate'] = edges['predicate'].astype('category')
edges['object_category'] = edges['object_category'].astype('category')
edges['edge_category'] = edges['edge_category'].astype('category')

# From unknown type categorical to known type categorical
edges = edges.categorize(columns=['subject_category','predicate','object_category','edge_category'])

# Prepare edge types and mappings
edge_types = edges['edge_category'].unique().compute()  # Compute edge types on scheduler

In [70]:
edges['subject_category'].cat.known

True

In [71]:
print(edges.dtypes)

id                    object
subject               object
subject_id             int64
subject_category    category
predicate           category
edge_category       category
object_category     category
object_id              int64
object                object
dtype: object

In [72]:
edges.head()

,id,subject,subject_id,subject_category,predicate,edge_category,object_category,object_id,object
0,uuid:6ef055fb-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,518846,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,517786,HGNC:989
1,uuid:6ef055fc-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,518846,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,517786,HGNC:989
2,uuid:a77a4a50-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,518846,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,517786,HGNC:989
3,uuid:c3ee0928-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,518846,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,517786,HGNC:989
4,uuid:c9de5a17-9bab-11ee-b780-6b2918cfaf31,HGNC:16391,518846,biolink:Gene,biolink:interacts_with,biolink:PairwiseGeneToGeneInteraction,biolink:Gene,517786,HGNC:989


In [43]:
edges[['subject_category','predicate','object_category','edge_category']].isna().any().compute()

subject_category    False
predicate           False
object_category     False
edge_category       False
dtype: bool

In [39]:
# Function to apply to each partition
def create_edge_tuple(df):
    # Ensure the function always returns an iterable, even if df is empty
    return zip(df['subject_category'], df['predicate'], df['object_category'],
               df['subject_id'], df['object_id'])

# Convert DataFrame to Bag of tuples
# display(edges)
ebag = edges.map_partitions(create_edge_tuple)
# display(list(ebag.compute()[0]))
ebag = ebag.to_bag()
# display(ebag.compute())


[None, None, None]

In [48]:
edges.npartitions

3

In [33]:
eb = ebag.compute()
eb

[None, None, None]

In [100]:
# Function to convert DataFrame rows to tuples for the Bag
def create_edge_tuple(row):
    return (row['subject_category'], row['predicate'], row['object_category']), row['subject_id'], row['object_id']

# Create a Bag from the DataFrame, and map the conversion function to each row
edges_bag = edges.map_partitions(lambda df: df.apply(create_edge_tuple, axis=1)).to_bag()
# display(edges_bag.compute())

In [101]:
# Key for the edge type mapping
def key_function(x):
    # print(x)
    return (x[0], x[1], x[2])

def binop(accumulator, edge):
    if edge is not None:  # Check if edge is not None
        # Append the tuple to the accumulator
        accumulator.append((edge[1], edge[2]))  # edge[1] is subject_id, edge[2] is object_id
    return accumulator

def combine(accumulator1, accumulator2):
    # print(accumulator1)
    # Extend the first accumulator with the second
    accumulator1.extend(accumulator2)
    return accumulator1

# # accumulate the edges for a specific key
# def binop(accumulator, edge):
#     # accumulator is a list of (subject_id, object_id) tuples
#     # edge is the current item being processed
#     accumulator.append([(x[3], x[4])])
#     return accumulator

# # combine multiple list that are accumulated
# def combine(accumulator1, accumulator2):
#     # Combine the lists from two accumulators
#     return accumulator1 + accumulator2

# Use foldby with the process_tuples function
edge_type_mappings = edges_bag.foldby(key=lambda x: x[0], binop=binop, combine=combine, initial=[])
edge_type_mappings = edge_type_mappings.compute()  # Trigger the computation


In [102]:
edge_type_mappings

[(('biolink:Gene', 'biolink:expressed_in', 'biolink:GrossAnatomicalStructure'),
  [(214529, 2302),
   (255135, 209),
   (521263, 145),
   (277734, 6514),
   (536392, 536397),
   (277996, 3848),
   (526349, 9248),
   (261180, 11324),
   (285549, 4639),
   (132180, 406),
   (523251, 537659),
   (541378, 550690),
   (284140, 1210),
   (156420, 14197),
   (554164, 553328),
   (288937, 4538),
   (523095, 557837),
   (189348, 11267),
   (302668, 4478),
   (11541, 9208),
   (520411, 19469),
   (148445, 16628),
   (288756, 1150),
   (527915, 3324),
   (551535, 553018),
   (287027, 3888),
   (148752, 1082),
   (266971, 232),
   (178614, 21467),
   (4330, 2456),
   (17698, 16215),
   (526489, 3324),
   (3342, 1464),
   (160026, 24583),
   (300647, 1207),
   (133935, 897),
   (542122, 542132),
   (147143, 28018),
   (11951, 8621),
   (221998, 466),
   (284153, 285257),
   (294300, 4639),
   (543835, 517892),
   (277169, 299077),
   (542301, 553690),
   (523875, 7833),
   (154858, 12446),
   (5262

In [19]:
################## INITIALIZE EDGES ##################

# Prepare edge types and mappings
edge_types = edges['edge_category'].unique()

# Function to apply to each partition
def create_edge_tuple(df):
    df['edge_key'] = list(zip(df['subject_category'], df['predicate'], df['object_category']))
    return df

# Key for the edge type mapping
def key_function(edge):
    return edge['edge_key']

# accumulate the edges for a specific key
def binop(accumulator, edge):
    # accumulator is a list of (subject_id, object_id) tuples
    # edge is the current item being processed
    accumulator.append([edge['subject_id'], edge['object_id']])
    return accumulator

# combine multiple list that are accumulated
def combine(accumulator1, accumulator2):
    # Combine the lists from two accumulators
    return accumulator1 + accumulator2


# Apply the function to each partition
_edf = edges.map_partitions(create_edge_tuple)
_edf = _edf[['edge_key','subject_id','object_id']]
_ebag = _edf.to_bag(format='dict')
# Using foldby to group by 'edge_key' and consolidate (subject_id, object_id) pairs
edge_type_mappings = _ebag.foldby(key=key_function, binop=binop, combine=combine, initial=[]).compute()    # Computation Heavy


[02/13/24 03:33:46] ERROR    Timed out during handshake while connecting to tcp://10.29.81.147:46004   ]8;id=957932;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py\utils.py]8;;\:]8;id=131226;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py#767\767]8;;\
                             after 30 s                                                                            
                             ╭────────────────── Traceback (most recent call last) ──────────────────╮             
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/comm/ │             
                             │ core.py:328 in connect                                                │             
                             │                                                                       │             
                             │   325 │   try:                                                        │             
                             │   326 │   │   # This would be better, but connections leak if worker  │             
                             │   327 │   │   # write, handshake = await asyncio.gather(comm.write(lo │             
                             │ ❱ 328 │   │   handshake = await asyncio.wait_for(comm.read(), time_le │             
                             │   329 │   │   await asyncio.wait_for(comm.write(local_info), time_lef │             
                             │   330 │   except Exception as exc:                                    │             
                             │   331 │   │   with suppress(Exception):                               │             
                             │                                                                       │             
                             │ /cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python │             
                             │ /3.8.10/lib/python3.8/asyncio/tasks.py:501 in wait_for                │             
                             │                                                                       │             
                             │   498 │   │   │   # after wait_for() returns.                         │             
                             │   499 │   │   │   # See https://bugs.python.org/issue32751            │             
                             │   500 │   │   │   await _cancel_and_wait(fut, loop=loop)              │             
                             │ ❱ 501 │   │   │   raise exceptions.TimeoutError()                     │             
                             │   502 │   finally:                                                    │             
                             │   503 │   │   timeout_handle.cancel()                                 │             
                             │   504                                                                 │             
                             ╰───────────────────────────────────────────────────────────────────────╯             
                             TimeoutError                                                                          
                                                                                                                   
                             The above exception was the direct cause of the following exception:                  
                                                                                                                   
                             ╭────────────────── Traceback (most recent call last) ──────────────────╮             
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils │             
                             │ .py:741 in wrapper                                                    │             
                             │                                                  

ERROR:tornado.application:Uncaught exception GET /info/call-stacks/tcp%3A%2F%2F10.29.81.147%3A46004.html (10.29.77.38)
HTTPServerRequest(protocol='http', host='localhost:8787', method='GET', uri='/info/call-stacks/tcp%3A%2F%2F10.29.81.147%3A46004.html', version='HTTP/1.1', remote_ip='10.29.77.38')
Traceback (most recent call last):
  File "/home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/comm/core.py", line 328, in connect
    handshake = await asyncio.wait_for(comm.read(), time_left())
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/asyncio/tasks.py", line 501, in wait_for
    raise exceptions.TimeoutError()
asyncio.exceptions.TimeoutError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/web.py", line 1704, in _execute
    result = await result
  File "/home/rahit/jupyter_py3/lib/python3.8/site-pa

[02/13/24 03:33:48] ERROR    Uncaught exception GET                                                     ]8;id=403101;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/web.py\web.py]8;;\:]8;id=571998;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/web.py#1789\1789]8;;\
                             /info/call-stacks/tcp%3A%2F%2F10.29.81.147%3A46004.html (10.29.77.38)                 
                             HTTPServerRequest(protocol='http', host='localhost:8787', method='GET',               
                             uri='/info/call-stacks/tcp%3A%2F%2F10.29.81.147%3A46004.html',                        
                             version='HTTP/1.1', remote_ip='10.29.77.38')                                          
                             ╭────────────────── Traceback (most recent call last) ───────────────────╮            
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/comm/c │            
                             │ ore.py:328 in connect                                                  │            
                             │                                                                        │            
                             │   325 │   try:                                                         │            
                             │   326 │   │   # This would be better, but connections leak if worker i │            
                             │   327 │   │   # write, handshake = await asyncio.gather(comm.write(loc │            
                             │ ❱ 328 │   │   handshake = await asyncio.wait_for(comm.read(), time_lef │            
                             │   329 │   │   await asyncio.wait_for(comm.write(local_info), time_left │            
                             │   330 │   except Exception as exc:                                     │            
                             │   331 │   │   with suppress(Exception):                                │            
                             │                                                                        │            
                             │ /cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/ │            
                             │ 3.8.10/lib/python3.8/asyncio/tasks.py:501 in wait_for                  │            
                             │                                                                        │            
                             │   498 │   │   │   # after wait_for() returns.                          │            
                             │   499 │   │   │   # See https://bugs.python.org/issue32751             │            
                             │   500 │   │   │   await _cancel_and_wait(fut, loop=loop)               │            
                             │ ❱ 501 │   │   │   raise exceptions.TimeoutError()                      │            
                             │   502 │   finally:                                                     │            
                             │   503 │   │   timeout_handle.cancel()                                  │            
                             │   504                                                                  │            
                             ╰────────────────────────────────────────────────────────────────────────╯            
                             TimeoutError                                                                          
                                                                                                                   
                             The above exception was the direct cause of the following exception:                  
                                                                                                                   
                             ╭────────────────── Traceback (most recent call last) ────────

[02/13/24 03:33:49] INFO     Event loop was unresponsive in Scheduler for 3.11s.  This is often caused  ]8;id=349500;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=682490;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#595\595]8;;\
                             by long-running GIL-holding functions or moving large chunks of data. This            
                             can cause timeouts and instability.                                                   

[02/13/24 03:34:01] ERROR    Timed out during handshake while connecting to tcp://10.29.81.147:46004   ]8;id=325488;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py\utils.py]8;;\:]8;id=98469;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py#767\767]8;;\
                             after 30 s                                                                            
                             ╭────────────────── Traceback (most recent call last) ──────────────────╮             
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/comm/ │             
                             │ core.py:328 in connect                                                │             
                             │                                                                       │             
                             │   325 │   try:                                                        │             
                             │   326 │   │   # This would be better, but connections leak if worker  │             
                             │   327 │   │   # write, handshake = await asyncio.gather(comm.write(lo │             
                             │ ❱ 328 │   │   handshake = await asyncio.wait_for(comm.read(), time_le │             
                             │   329 │   │   await asyncio.wait_for(comm.write(local_info), time_lef │             
                             │   330 │   except Exception as exc:                                    │             
                             │   331 │   │   with suppress(Exception):                               │             
                             │                                                                       │             
                             │ /cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python │             
                             │ /3.8.10/lib/python3.8/asyncio/tasks.py:501 in wait_for                │             
                             │                                                                       │             
                             │   498 │   │   │   # after wait_for() returns.                         │             
                             │   499 │   │   │   # See https://bugs.python.org/issue32751            │             
                             │   500 │   │   │   await _cancel_and_wait(fut, loop=loop)              │             
                             │ ❱ 501 │   │   │   raise exceptions.TimeoutError()                     │             
                             │   502 │   finally:                                                    │             
                             │   503 │   │   timeout_handle.cancel()                                 │             
                             │   504                                                                 │             
                             ╰───────────────────────────────────────────────────────────────────────╯             
                             TimeoutError                                                                          
                                                                                                                   
                             The above exception was the direct cause of the following exception:                  
                                                                                                                   
                             ╭────────────────── Traceback (most recent call last) ──────────────────╮             
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils │             
                             │ .py:741 in wrapper                                                    │             
                             │                                                   

...

[02/13/24 03:53:19] INFO     Register worker <WorkerState 'tcp://10.29.85.1:46589', name:         ]8;id=736070;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=257631;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4212\4212]8;;\
                             SLURMCluster-2, status: init, memory: 0, processing: 0>                               

                    INFO     Starting worker compute stream, tcp://10.29.85.1:46589               ]8;id=335050;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=871427;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5468\5468]8;;\

                    INFO     Starting established connection to tcp://10.29.85.1:38812                  ]8;id=293051;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=36447;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#867\867]8;;\

[02/13/24 03:55:07] ERROR    Task exception was never retrieved                                 ]8;id=56156;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/asyncio/base_events.py\base_events.py]8;;\:]8;id=199689;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/asyncio/base_events.py#1707\1707]8;;\
                             future: <Task finished name='Task-405998'                                             
                             coro=<Client._gather.<locals>.wait() done, defined at                                 
                             /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/cl                    
                             ient.py:2134> exception=AllExit()>                                                    
                             ╭────────────── Traceback (most recent call last) ───────────────╮                    
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distribute │                    
                             │ d/client.py:2143 in wait                                       │                    
                             │                                                                │                    
                             │   2140 │   │   │   else:                                       │                    
                             │   2141 │   │   │   │   await st.wait()                         │                    
                             │   2142 │   │   │   if st.status != "finished" and errors == "r │                    
                             │ ❱ 2143 │   │   │   │   raise AllExit()                         │                    
                             │   2144 │   │                                                   │                    
                             │   2145 │   │   while True:                                     │                    
                             │   2146 │   │   │   logger.debug("Waiting on futures to clear b │                    
                             ╰────────────────────────────────────────────────────────────────╯                    
                             AllExit                                                                               

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:33                                                                                   │
│                                                                                                  │
│   30 _edf = _edf[['edge_key','subject_id','object_id']]                                          │
│   31 _ebag = _edf.to_bag(format='dict')                                                          │
│   32 # Using foldby to group by 'edge_key' and consolidate (subject_id, object_id) pairs         │
│ ❱ 33 edge_type_mappings = _ebag.foldby(key=key_function, binop=binop, combine=combine, initia    │
│   34                                                                                             │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/base.py:314 in compute                  │
│                                                                                                  │
│    311 │   │   --------                                                                          │
│    312 │   │   dask.base.compute                                                                 │
│    313 │   │   """                                                                               │
│ ❱  314 │   │   (result,) = compute(self, traverse=False, **kwargs)                               │
│    315 │   │   return result                                                                     │
│    316 │                                                                                         │
│    317 │   def __await__(self):                                                                  │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/base.py:599 in compute                  │
│                                                                                                  │
│    596 │   │   keys.append(x.__dask_keys__())                                                    │
│    597 │   │   postcomputes.append(x.__dask_postcompute__())                                     │
│    598 │                                                                                         │
│ ❱  599 │   results = schedule(dsk, keys, **kwargs)                                               │
│    600 │   return repack([f(r, *a) for r, (f, a) in zip(results, postcomputes)])                 │
│    601                                                                                           │
│    602                                                                                           │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/client.py:3137 in get            │
│                                                                                                  │
│   3134 │   │   │   │   except Exception:                                                         │
│   3135 │   │   │   │   │   should_rejoin = False                                                 │
│   3136 │   │   │   try:                                                                          │
│ ❱ 3137 │   │   │   │   results = self.gather(packed, asynchronous=asynchronous, direct=direct)   │
│   3138 │   │   │   finally:                                                                      │
│   3139 │   │   │   │   for f in futures.values():                                                │
│   3140 │   │   │   │   │   f.release()                                                           │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distrib

In [111]:
import sys

# Assuming 'result' is your Dask-computed variable
size_in_bytes = sys.getsizeof(data)
print(f"Size of the variable in bytes: {size_in_bytes}")

# Convert bytes to more readable units like MB
size_in_mb = size_in_bytes / (1024**2)
print(f"Size of the variable in MB: {size_in_mb}")


Size of the variable in bytes: 48

Size of the variable in MB: 4.57763671875e-05

In [95]:
import pickle

with open('edge_type_mappings.pkl', 'wb') as f:
    pickle.dump(edge_type_mappings, f)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 import pickle                                                                                │
│   2                                                                                              │
│   3 with open('edge_type_mappings.pkl', 'wb') as f:                                              │
│ ❱ 4 │   pickle.dump(edge_type_mappings, f)                                                       │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
PicklingError: Can't pickle <function <lambda> at 0x2b05f404c550>: attribute lookup <lambda> on __main__ failed

[02/15/24 03:47:05] ERROR    'NoneType' object has no attribute 'add_next_tick_callback'               ]8;id=42227;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py\utils.py]8;;\:]8;id=158111;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py#767\767]8;;\
                             ╭────────────────── Traceback (most recent call last) ──────────────────╮             
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils │             
                             │ .py:741 in wrapper                                                    │             
                             │                                                                       │             
                             │    738 │   │   │                                                      │             
                             │    739 │   │   │   async def wrapper(*args, **kwargs):                │             
                             │    740 │   │   │   │   with self:                                     │             
                             │ ❱  741 │   │   │   │   │   return await func(*args, **kwargs)         │             
                             │    742 │   │                                                          │             
                             │    743 │   │   else:                                                  │             
                             │    744                                                                │             
                             │                                                                       │             
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/dashb │             
                             │ oard/components/shared.py:314 in cb                                   │             
                             │                                                                       │             
                             │   311 │   │   │   │   metadata = None                                 │             
                             │   312 │   │   │   if isinstance(prof, gen.Future):                    │             
                             │   313 │   │   │   │   prof, metadata = await asyncio.gather(prof, met │             
                             │ ❱ 314 │   │   │   self.doc().add_next_tick_callback(lambda: self.upda │             
                             │   315 │   │                                                           │             
                             │   316 │   │   self.server.loop.add_callback(cb)                       │             
                             │   317                                                                 │             
                             ╰───────────────────────────────────────────────────────────────────────╯             
                             AttributeError: 'NoneType' object has no attribute                                    
                             'add_next_tick_callback'                                                              

2024-02-15 03:47:05,435 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x2b05f41bb730>>, <Task finished name='Task-6068326' coro=<ProfileTimePlot.trigger_update.<locals>.cb() done, defined at /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py:739> exception=AttributeError("'NoneType' object has no attribute 'add_next_tick_callback'")>)
Traceback (most recent call last):
  File "/home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/ioloop.py", line 765, in _discard_future_result
    future.result()
  File "/home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py", line 741, in wrapper
    return await func(*args, **kwargs)
  File "/home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/dashboard/co

                    ERROR    Exception in callback functools.partial(<bound method                    ]8;id=434843;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/ioloop.py\ioloop.py]8;;\:]8;id=854341;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/ioloop.py#761\761]8;;\
                             IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object               
                             at 0x2b05f41bb730>>, <Task finished name='Task-6068326'                               
                             coro=<ProfileTimePlot.trigger_update.<locals>.cb() done, defined at                   
                             /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/utils.py              
                             :739> exception=AttributeError("'NoneType' object has no attribute                    
                             'add_next_tick_callback'")>)                                                          
                             ╭───────────────── Traceback (most recent call last) ──────────────────╮              
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/ioloop.p │              
                             │ y:741 in _run_callback                                               │              
                             │                                                                      │              
                             │   738 │   │      CancelledErrors are no longer logged.               │              
                             │   739 │   │   """                                                    │              
                             │   740 │   │   try:                                                   │              
                             │ ❱ 741 │   │   │   ret = callback()                                   │              
                             │   742 │   │   │   if ret is not None:                                │              
                             │   743 │   │   │   │   from tornado import gen                        │              
                             │   744                                                                │              
                             │                                                                      │              
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/tornado/ioloop.p │              
                             │ y:765 in _discard_future_result                                      │              
                             │                                                                      │              
                             │   762 │                                                              │              
                             │   763 │   def _discard_future_result(self, future: Future) -> None:  │              
                             │   764 │   │   """Avoid unhandled-exception warnings from spawned cor │              
                             │ ❱ 765 │   │   future.result()                                        │              
                             │   766 │                                                              │              
                             │   767 │   def split_fd(                                              │              
                             │   768 │   │   self, fd: Union[int, _Selectable]                      │              
                             │                                                                      │              
                             │ /home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/util │              
                             │ s.py:741 in wrapper                                                  │              
                             │                                                        

[02/15/24 03:48:17] WARNING  Worker failed to heartbeat within 300 seconds. Closing: <WorkerState ]8;id=916906;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=819106;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#7716\7716]8;;\
                             'tcp://10.29.85.254:33416', name: SLURMCluster-5, status: running,                    
                             memory: 0, processing: 0>                                                             

...

2024-02-15 03:48:17,663 - distributed.scheduler - ERROR - Couldn't gather keys {"('foldby-b-b1c8781001549f00d8728f67e5dbce19', 0)": ['tcp://10.29.85.254:33416']} state: [None] workers: ['tcp://10.29.85.254:33416']
NoneType: None


                    ERROR    Couldn't gather keys {"('foldby-b-b1c8781001549f00d8728f67e5dbce19', ]8;id=371654;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=31707;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5698\5698]8;;\
                             0)": ['tcp://10.29.85.254:33416']} state: [None] workers:                             
                             ['tcp://10.29.85.254:33416']                                                          
                             NoneType: None                                                                        

2024-02-15 03:48:17,672 - distributed.scheduler - ERROR - Shut down workers that don't have promised key: ['tcp://10.29.85.254:33416'], ('foldby-b-b1c8781001549f00d8728f67e5dbce19', 0)
NoneType: None


                    ERROR    Shut down workers that don't have promised key:                      ]8;id=179457;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=278533;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#5716\5716]8;;\
                             ['tcp://10.29.85.254:33416'],                                                         
                             ('foldby-b-b1c8781001549f00d8728f67e5dbce19', 0)                                      
                             NoneType: None                                                                        

[02/15/24 03:48:34] INFO     Connection to tcp://10.29.85.254:41074 has been closed.                    ]8;id=360108;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=875328;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

In [104]:
# Add edges to the graph
for edge_key, edge_indices in edge_type_mappings:
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    data[edge_key].edge_index = edge_index

In [109]:
len(data.edge_types)

43

In [113]:
# data.save('test.pt')
torch.save(data, 'test_hetero_data.pt')

In [21]:
# Prepare node mapping and node types
node_mapping = {node_id: i for i, node_id in enumerate(nodes_df['id'].unique())}
node_types = nodes_df['category'].unique()

# Initialize HeteroData for the heterogeneous graph
data = HeteroData()

# Add nodes to the graph
for node_type in node_types:
    # get nodes of type `node_type`
    mask = nodes_df['category'] == node_type
    type_nodes = nodes_df[mask]
    
    # node_features = torch.ones((type_nodes.shape[0], 1))
    data[node_type].num_nodes = type_nodes.shape[0] # node_features # torch.tensor(type_nodes.index.to_list(), dtype=torch.long).view(len(type_nodes.index.to_list()),1)

    # Add dummy features (e.g., a simple constant feature)
    data[node_type].x = torch.ones((data[node_type].num_nodes, 1))  # Each user has a feature vector of [1]


# Prepare edge types and mappings
edge_types = edges_df['edge_category'].unique()
edge_type_mappings = defaultdict(list)

for _, row in edges_df.iterrows():
    subject_type = nodes_df.loc[nodes_df['id'] == row['subject'], 'category'].values[0]
    object_type = nodes_df.loc[nodes_df['id'] == row['object'], 'category'].values[0]
    edge_key = (row['subject_category', row['predicate'], row['object_category'])
    edge_type_mappings[edge_key].append((node_mapping[row['subject']], node_mapping[row['object']]))

# Add edges to the graph
for edge_key, edge_indices in edge_type_mappings.items():
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    data[edge_key].edge_index = edge_index


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/pandas/core/indexes/base.py:3621 in get_loc  │
│                                                                                                  │
│   3618 │   │   │   │   )                                                                         │
│   3619 │   │   │   casted_key = self._maybe_cast_indexer(key)                                    │
│   3620 │   │   │   try:                                                                          │
│ ❱ 3621 │   │   │   │   return self._engine.get_loc(casted_key)                                   │
│   3622 │   │   │   except KeyError as err:                                                       │
│   3623 │   │   │   │   raise KeyError(key) from err                                              │
│   3624 │   │   │   except TypeError:                                                             │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:136                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:163                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5198                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5206                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'id'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:26                                                                                   │
│                                                                                                  │
│   23 edge_type_mappings = defaultdict(list)                                                      │
│   24                                                                                             │
│   25 for _, row in edges_df.iterrows():                                                          │
│ ❱ 26 │   subject_type = nodes_df.loc[nodes_df['id'] == row['subject'], 'category'].values[0]     │
│   27 │   object_type = nodes_df.loc[nodes_df['id'] == row['object'], 'category'].values[0]       │
│   28 │   edge_key = (subject_type, row['predicate'], object_type)                                │
│   29 │   edge_type_mappings[edge_key].append((node_mapping[row['subject']], node_mapping[row[    │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/dask/dataframe/core.py:4624 in __getitem__   │
│                                                                                                  │
│   4621 │   │   │   │   │   return self.loc[key]                                                  │
│   4622 │   │   │                                                                                 │
│   4623 │   │   │   # error is raised from pandas                                                 │
│ ❱ 4624 │   │   │   meta = self._meta[_extract_meta(key)]                                         │
│   4625 │   │   │   dsk = partitionwise_graph(operator.getitem, name, self, key)                  │
│   4626 │   │   │   graph = HighLevelGraph.from_collections(name, dsk, dependencies=[self])       │
│   4627 │   │   │   return new_dd_object(graph, name, meta, self.divisions)

[02/07/24 22:36:40] INFO     Connection to tcp://10.29.82.239:34758 has been closed.                    ]8;id=28316;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=823804;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#877\877]8;;\

                    INFO     Remove worker <WorkerState 'tcp://10.29.82.239:36343', name:         ]8;id=980790;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=114701;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4795\4795]8;;\
                             SLURMCluster-0, status: running, memory: 0, processing: 0>                            

                    INFO     Removing comms to tcp://10.29.82.239:36343                                ]8;id=412105;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py\core.py]8;;\:]8;id=294694;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/core.py#1480\1480]8;;\

                    INFO     Lost all workers                                                     ]8;id=729680;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=647358;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#4875\4875]8;;\

In [ ]:
len(edge_types)
edge_type_mappings

In [82]:
cluster.close()

[02/15/24 03:03:01] INFO     Scheduler closing...                                                 ]8;id=685697;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=628270;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3925\3925]8;;\

                    INFO     Scheduler closing all comms                                          ]8;id=903181;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=57427;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3947\3947]8;;\

In [83]:
client.close()